# Random Forests

---

*Features*

- Grid search on various representations and mtry (max depth) values a random forest of 500 trees with max leaf nodes = 99
- Save results into a CSV file for later study; see next notebook for results

## Setup & Load Data

In [1]:
import os
import time
import json
import urlextract
import numpy as np
import pandas as pd

from datetime import datetime
from nltk.stem import WordNetLemmatizer

dt_object = datetime.fromtimestamp(time.time())
day, T = str(dt_object).split('.')[0].split(' ')
print('Revised on: ' + day)

def load_data(data):
    raw_path = os.path.join("..","data","1_raw")
    filename = ''.join([data, ".csv"])
    out_dfm = pd.read_csv(os.path.join(raw_path, filename))
    out_arr = np.array(out_dfm.iloc[:,0].ravel())
    return out_arr

X_train = load_data("X_train")
y_train = load_data("y_train")

y = y_train.copy()

# transform y_array into int type
y[y=='ham'] = 0
y[y=='spam'] = 1
y = y.astype('int')

# load contractions map for custom cleanup
with open("contractions_map.json") as f:
    contractions_map = json.load(f)

Revised on: 2020-12-30


## BoT & Tfidf

In [2]:
import custom.clean_preprocess as cp
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

pipe = Pipeline([('counter', cp.DocumentToNgramCounterTransformer(n_grams=3)),
                 ('bot', cp.WordCounterToVectorTransformer(vocabulary_size=2000)),
                 ('tfidf', TfidfTransformer(sublinear_tf=True))
                ])

X_counter = pipe['counter'].fit_transform(X_train)
X_bot = pipe['bot'].fit_transform(X_counter)
X_tfidf = pipe.fit_transform(X_train)

### SVD

In [3]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from sklearn.utils.extmath import svd_flip

def perform_SVD(X, n_components=300):
    X_array = X.asfptype()
    U, Sigma, VT = svds(X_array.T, k=n_components)
    # reverse outputs
    Sigma = Sigma[::-1]
    U, VT = svd_flip(U[:, ::-1], VT[::-1])
    # return V 
    V = VT.T
    return V

X_svd_bot = perform_SVD(X_bot)
X_svd_tfidf = perform_SVD(X_tfidf) # Intro to Text Analytics with R approach

## Cosine Similarity

In [4]:
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

X_cossim_svd_bot = cosine_similarity(X_svd_bot)
X_cossim_svd_tfidf = cosine_similarity(X_svd_tfidf)

train_df = pd.DataFrame({'sms':X_train, 'target':y_train})

# get spam indexes
spam_ix = train_df.loc[train_df['target']=='spam'].index

# calculate average spam similarity on SVD
mean_spam_sims_bot, mean_spam_sims_tfidf = [], []

for ix in range(X_cossim_svd_bot.shape[0]):
    mean_spam_sims_bot.append(np.mean(X_cossim_svd_bot[ix, spam_ix]))
    mean_spam_sims_tfidf.append(np.mean(X_cossim_svd_tfidf[ix, spam_ix]))

# There are four options:
# 1. cossim on bot + bot: hm, maybe? Let's test it out
# 2. cossim on tfidf + bot: this is just weird
# 3. cossim on bot + tfidf: this is just weird
# 4. cossim on tfidf + tfidf: Intro to Text Analytics with R approach
X_bot_cossim_bot = sp.hstack((csr_matrix(mean_spam_sims_bot).T, X_bot)) # option 1
X_tfidf_cossim_tfidf = sp.hstack((csr_matrix(mean_spam_sims_tfidf).T, X_tfidf)) # option 4

## Feature Engineering

In [5]:
%%capture output

# custom feature engineering module
import custom.feature_engineering as Fe

try:
    clean_train_docs, X_train_feat = Fe.DocumentToFeaturesCounterTransformer().fit_transform(X_train)
except Warning as e:
    pass # avoids RuntimeErrors because of divisions by zero in calculating means/stds

# impute with zeros and remove RSR
X_train_feat[np.isnan(X_train_feat)] = 0
X_feat = X_train_feat[:,:6] 

# stack with BoT and SVD of Tfidf
X_bot_feat = sp.hstack((X_bot, X_feat))
X_svd_feat = sp.hstack((csr_matrix(X_svd_tfidf), X_feat))

## Random Forests

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, recall_score

def scikitlearn_cv(clf, X, y, seed_, cv=5, test_size=.25):
    
    scorer_ = {
        'acc': make_scorer(accuracy_score),
        'tpr': make_scorer(recall_score, pos_label=1),
        'tnr': make_scorer(recall_score, pos_label=0)
    }
    
    acc = cross_val_score(clf, X, y, cv=cv, verbose=0, scoring=scorer_['acc'], n_jobs=-1)
    tpr = cross_val_score(clf, X, y, cv=cv, verbose=0, scoring=scorer_['tpr'], n_jobs=-1)
    tnr = cross_val_score(clf, X, y, cv=cv, verbose=0, scoring=scorer_['tnr'], n_jobs=-1)
    
    return acc.mean(), tpr.mean(), tnr.mean()

def collect_cvs(clf, Xs, Xnames, y, seed_, cv=10, test_size=.25):

    accs, tprs, tnrs, secs = [], [], [], []
    for X in Xs:
        start_cv = time.time()
        acc, tpr, tnr = scikitlearn_cv(clf, X, y, seed_=seed_, cv=cv, test_size=test_size)
        accs.append(round(acc, 4))
        tprs.append(round(tpr, 4))
        tnrs.append(round(tnr, 4))
        secs.append(round(time.time() - start_cv, 1))

    data = {'Representation': Xnames,
            'mean_accuracy': accs,
            'mean_sensitivity': tprs, 
            'mean_specificity': tnrs,
            'elapsed_seconds':secs
           }
    
    return data

def build_random_forests(Xs, Xnames, y, cv_seed, rf_seed, mtry_, trees, 
                         max_leaf_nodes, cv=5, max_samples=None, n_jobs=-1):
    """Given:
           Xs: a list of X representations (training data)
           Xnames: a list their names (descriptions)
           y: the target variable
           cv_seed: random seed for cross validation
           rf_seed: random seed for rf classifier
           mtry_: a list of values for the max_features param
           trees: number of trees
           max_leaf_nodes: max number of leaf nodes
           cv: number of folds, defaults to k=5
           max_samples: max num of samples, defaults to None
           n_jobs: defaults to -1 (all cores but one)
       Return:
           A dataframe of results of cv over various mtry values
           With mean accuracy, sensitivity, specificity
    """
    list_of_dfs = []
    for mtry in mtry_:
        rf_clf = RandomForestClassifier(n_estimators=trees,
                                        max_samples=None,
                                        max_features=mtry,
                                        max_leaf_nodes=max_leaf_nodes,
                                        random_state=rf_seed,
                                        n_jobs=n_jobs,
                                        verbose=0)
        
        data = collect_cvs(rf_clf, Xs, Xnames, y, seed_=cv_seed, cv=cv)
        df = pd.DataFrame(data)
        df['mtry'] = mtry
        
        list_of_dfs.append(df)
     
    flattened_df = pd.concat(list_of_dfs)
    
    # reset index
    ix_num = len(mtry_) * len(Xs)
    flattened_df.index = range(ix_num)
    
    return flattened_df

## Custom Gridsearches

10-fold CV with on random forest with 500 trees and 99 max leaf nodes on all 7 representations - varying mtry values.

In [7]:
Xs = [
    X_bot, 
    X_tfidf, 
    X_svd_bot,
    X_svd_tfidf,
    X_bot_feat,
    X_svd_feat,
    X_bot_cossim_bot, 
    X_tfidf_cossim_tfidf
]

Xnames = [
    'BoT', 
    'Tfidf', 
    'SVD on BoT',
    'SVD on Tfidf', 
    'BoT + features',
    'SVD + features',
    'Cossim on BoT',
    'Cossim on Tfidf'
]

In [8]:
cv_data1 = build_random_forests(Xs, 
                                Xnames, 
                                y,
                                cv_seed=423, 
                                rf_seed=514,
                                mtry_=[10, 25, 50, 100, 250, None],
                                trees=500, 
                                max_leaf_nodes=99, 
                                cv=5)

In [9]:
cv_data1

,Representation,mean_accuracy,mean_sensitivity,mean_specificity,elapsed_seconds,mtry
0,BoT,0.9708,0.7795,1.0000,6.9,10
1,Tfidf,0.9708,0.7795,1.0000,5.8,10
2,SVD on BoT,0.9641,0.7369,0.9988,49.1,10
3,SVD on Tfidf,0.9579,0.6846,0.9997,43.3,10
4,BoT + features,0.9695,0.7698,1.0000,6.7,10
5,SVD + features,0.9674,0.7543,1.0000,55.2,10
6,Cossim on BoT,0.9754,0.8143,1.0000,6.5,10
7,Cossim on Tfidf,0.9777,0.8316,1.0000,7.2,10
8,BoT,0.9810,0.8587,0.9997,7.5,25
9,Tfidf,0.9815,0.8626,0.9997,8.5,25


In [11]:
dir_path = os.path.join("..","data","3_modeling")
try:
    os.stat(dir_path)
except:
    os.mkdir(dir_path)

In [12]:
file_path = os.path.join(dir_path, "cv_data1.csv")
cv_data1.to_csv(file_path, index=False)

---